### 0. Installing Dependencies

In [56]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


### 1. Test Random Environment with PettingZoo

In [21]:
from pettingzoo.mpe import simple_v2
import numpy as np

In [54]:
parallel_env = simple_v2.parallel_env(max_cycles=30, continuous_actions=False)
observations = parallel_env.reset()

agent = parallel_env.agents[0]

observation_space = parallel_env.observation_space(agent)
action_space = parallel_env.action_space(agent)

parallel_env.observation_space = observation_space
parallel_env.action_space = action_space


In [33]:
episodes = 10
for episode in range(1, episodes + 1):
    state = parallel_env.reset()[agent]
    done = False
    score = 0

    while not done:
        action = np.random.randint(0, action_space.n)
        n_state, reward, _, done, _ = parallel_env.step({agent: action})
        n_state = n_state[agent]
        reward = reward[agent]
        done = done[agent]

        score += reward
        state = n_state

    print("Episode:{} Score:{}".format(episode, score))


Episode:1 Score:-38.75532836783367
Episode:2 Score:-103.92529846244271
Episode:3 Score:-75.43644400411934
Episode:4 Score:-12.102438417653616
Episode:5 Score:-147.91176842037106
Episode:6 Score:-28.089079712596412
Episode:7 Score:-73.36012172863425
Episode:8 Score:-102.08646601151612
Episode:9 Score:-56.77522873301166
Episode:10 Score:-1.6591935670257663


### 2. Create Deep Learning Model with Keras

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

2023-03-09 19:31:19.626470: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [37]:
def build_model(learning_rate, action_space):
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + observation_space.shape))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(action_space.n, activation="linear"))
    model.compile(loss="mse", optimizer=Adam(lr=learning_rate))
    return model


In [48]:
del model

In [49]:
model = build_model(0.001, action_space)

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 5)                 125       
                                                                 
Total params: 845
Trainable params: 845
Non-trainable params: 0
_________________________________________________________________


### 3. Build Agent with Keras-RL

In [51]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [52]:
def build_agent(model, action_space):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(
        model=model,
        memory=memory,
        policy=policy,
        nb_actions=action_space.n,
        nb_steps_warmup=10,
        target_model_update=1e-2,
    )
    return dqn


In [55]:
dqn = build_agent(model, action_space)
dqn.compile(Adam(learning_rate=1e-3), metrics=["mae"])
dqn.fit(parallel_env, nb_steps=50000, visualize=False, verbose=1)


2023-03-09 19:50:46.414347: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_5_2/kernel/Assign' id:640 op device:{requested: '', assigned: ''} def:{{{node dense_5_2/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5_2/kernel, dense_5_2/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


AttributeError: 'Adam' object has no attribute '_name'